In [1]:
import datetime

current_time = datetime.datetime.now()
print(current_time)

2024-12-07 14:19:41.590424


In [44]:
import bokeh 
bokeh.__version__

'3.5.1'

In [ ]:
# Load libraries 
## Raw Packages
import numpy as np
import pandas as pd
from math import pi
# from pandas_datareader import data as pdr
import datetime, time
## Data Source
import yfinance as yf
# To let pdr get data from yahoo -- https://stackoverflow.com/questions/74912452/typeerror-string-indices-must-be-integer-pandas-datareader
# yf.pdr_override()
from yahoo_fin import stock_info as si 
## Data Visualization 
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import column, row
# Site Scrapping
import requests
import json
import urllib.request
# Utilities
from IPython.display import clear_output

- Web Sources

https://medium.com/c%C3%B3digo-ecuador/python-web-scraping-yahoo-finance-stock-dividend-history-d9084c85c805

http://theautomatic.net/yahoo_fin-documentation/#get_company_info

https://github.com/shilewenuw/get_all_tickers/issues/12

https://towardsdatascience.com/parse-thousands-of-stock-recommendations-in-minutes-with-python-6e3e562f156d

https://towardsdatascience.com/python-how-to-get-live-market-data-less-than-0-1-second-lag-c85ee280ed93

https://pypi.org/project/yfinance/

In [3]:
#########################################################################
# Functions

apiBase = 'https://query2.finance.yahoo.com'
headers = { 
  "User-Agent": 
  "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"
}

def get_credentials(cookieUrl='https://fc.yahoo.com', crumbUrl=apiBase+'/v1/test/getcrumb'):
  '''
  https://stackoverflow.com/questions/76065035/yahoo-finance-v7-api-now-requiring-cookies-python
  '''
  cookie = requests.get(cookieUrl).cookies
  crumb = requests.get(url=crumbUrl, cookies=cookie, headers=headers).text
  return {'cookie': cookie, 'crumb': crumb}

def get_quote(symbols, credentials):
  '''
  https://stackoverflow.com/questions/76065035/yahoo-finance-v7-api-now-requiring-cookies-python
  '''
  final_list = []
  try:
    url = apiBase + '/v7/finance/quote'
    symbols_a = np.array_split(symbols, 3)
    for sp in symbols_a:
      params = {'symbols': ','.join(sp), 'crumb': credentials['crumb']}
      response = requests.get(url, params=params, cookies=credentials['cookie'], headers=headers)
      quotes = response.json()['quoteResponse']['result']
      final_list  = final_list + quotes
    return final_list
  except: 
    pass

def info_parser(df, quot, col, element):
      try: 
        df.loc[:, col] = quot[element]
      except: 
        df.loc[:, col] = '' 
      return df 

def get_tickers_information(credentials, tickers, get_quote):
    col_element = {'Price':'regularMarketPrice', 
               'Divident_Yield' : 'dividendYield',
               'Analyst_Recomdtn': 'averageAnalystRating',
               'Divident_Rate' : 'dividendRate'
               }
    quotes = get_quote(tickers, credentials)
    stock_info = pd.DataFrame()
    for quote in quotes:
        try: 
            print(f"{quote['symbol']}") # price is {quote['currency']} {quote['regularMarketPrice']} {quote['dividendYield']}")
            tickerdiv = si.get_dividends(quote['symbol'])
            tickerdiv = tickerdiv[tickerdiv.index == tickerdiv.index.max()].reset_index()
            tickerdiv.rename(columns = {'index':'Dividend_DT', 'ticker':'Symbol', 'dividend':'Dividend_AMT'}, inplace = True)
            for col, element in col_element.items():
                info_parser(tickerdiv, quote, col, element)
            stock_info = pd.concat([stock_info, tickerdiv], axis = 0)
        except Exception as E: 
            print("\t", E)
            continue
    stock_info[['Symbol', 'Price', 'Dividend_AMT', 'Dividend_DT', 'Divident_Yield', 'Divident_Rate', 'Analyst_Recomdtn']].reset_index(drop = True)
    stock_info.columns = [col.upper() for col in stock_info.columns]
    return stock_info



def min_value(value, df, field):
    if value == None:
        value = df[field].min()
    return value

def max_value(value, df, field):
    if value == None:
        value = df[field].max()
    return value

def filter_stocks(df, minprice, maxprice, minpct, maxpct, country):
    if 'RS_Rating' in df.columns:
        df = df[df.RS_Rating.isna()==False]
    return df[(df.LAST_PRICE >= minprice)\
                 &(df.LAST_PRICE <= maxprice)\
                 &(df.PCHANGE >= minpct)\
                 &(df.PCHANGE <= maxpct)\
                 &(df.COUNTRY == country)]\
            .sort_values(by = ['SYMBOL', 'LAST_PRICE', 'PCHANGE'], ascending = [True, False, True])

In [4]:
def get_nasdaqapi(exchange = None):
    '''
    Get the latest value of all stocks FROM Nasdaq.com
    https://github.com/shilewenuw/get_all_tickers/issues/12
    
    Available exchanges: nasdaq, amex, nyse
    '''
    headers = {'authority': 'api.nasdaq.com',
                'accept': 'application/json, text/plain, */*',
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
                'origin': 'https://www.nasdaq.com',
                'sec-fetch-site': 'same-site',
                'sec-fetch-mode': 'cors',
                'sec-fetch-dest': 'empty',
                'referer': 'https://www.nasdaq.com/',
                'accept-language': 'en-US,en;q=0.9',
                }
    dparams = (('tableonly', 'true'),
                ('limit', '250'),
                ('offset', '0'),
                ('download', 'true')
             )    
    if exchange != None:
        dparams = dparams + (('exchange',exchange),)
    r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers = headers, params = dparams)
    stocksj = r.json()['data']
    stocksdf = pd.DataFrame(stocksj['rows'], columns = stocksj['headers'])
    stocksdf = stocksdf[~stocksdf['symbol'].str.contains(r"\.|\^")]
    # display(stocks.shape)
    # Changing values to numbers in last price and percent change fields. 
    stocksdf.loc[:,'last_price'] = stocksdf.lastsale.str.replace(r'\$', '', regex = True).astype(float)
    stocksdf.loc[:,'pchange'] = stocksdf.pctchange.str.replace(r'\%|\s', '', regex = True)#.astype(float)
    stocksdf.loc[:,'symbol'] = stocksdf.symbol.str.replace(r'\/', '-', regex = True)#.astype(float)

    stocksdf.loc[:,'pchange'] = stocksdf.pchange.str.replace(r'^$', '-999', regex = True).astype(float)
    stocksdf.rename(columns = {'symbol':'Symbol'}, inplace = True)
    stocksdf.loc[:,'Symbol'] = stocksdf['Symbol'].str.strip()
    stocksdf.columns = [col.upper() for col in stocksdf.columns]

    return stocksdf#, stocksj

In [5]:
# ft100 = si.tickers_ftse100()
ft100= pd.read_html("https://en.wikipedia.org/wiki/FTSE_100_Index", attrs = {"id": "constituents"})[0].Ticker.unique()

ft100 = [i.replace(r'.', '-') for i in ft100]
# ft250 = si.tickers_ftse250()
ft250 = pd.read_html("https://en.wikipedia.org/wiki/FTSE_250_Index", attrs = {"id": "constituents"})[0].Ticker.unique()

ft250 = [i.replace(r'.', '-') for i in ft250]
ibo = si.tickers_ibovespa()
ibo = [i.replace(r'.', '-') for i in ibo]
n50 = si.tickers_nifty50()
n50 = [i.replace(r'.', '-') for i in n50]
nfb = si.tickers_niftybank()
nfb = [i.replace(r'.', '-') for i in nfb]
nas = si.tickers_nasdaq()
nas = [i.replace(r'.', '-') for i in nas]
dow = si.tickers_dow()
dow = [i.replace(r'.', '-') for i in dow]
othr = si.tickers_other()
othr = [i.replace(r'.', '-') for i in othr]
sp500 = si.tickers_sp500()
sp500 = [i.replace(r'.', '-') for i in sp500]


all_yh_tic = {'ft100':ft100, 'ft250': ft250, 'ibo':ibo, 'n50':n50, 'nfb':nfb, 'nas':nas, 'dow':dow, 'other':othr, 'sp500': sp500}
for key, value in all_yh_tic.items():
    print(key, '\t', len(value))
    
distinct_tickers = list(set(sp500) | set(dow)| set(nas)|set(othr)|set(ft100)| set(ft250)|set(ibo)|set(n50)|set(nfb))
print('Total Distinct Tickers',len(distinct_tickers))

/opt/anaconda3/envs/p312/lib/python3.12/site-packages/yahoo_fin/stock_info.py:235: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(requests.get(site, headers=headers).text)[0]


ft100 	 100
ft250 	 249
ibo 	 64
n50 	 29
nfb 	 12
nas 	 4808
dow 	 30
other 	 6342
sp500 	 503
Total Distinct Tickers 11511


In [6]:
# Collect ingformation from NASDAQ 
nasdaq = get_nasdaqapi()
all_stocks = get_quote(distinct_tickers, get_credentials())


In [7]:
stock_df = pd.DataFrame()
stocks_sp = np.array_split(all_stocks, 20)
for i, sp in enumerate(stocks_sp):
    for num, st in enumerate(sp):
        if num%5 == 0: 
            clear_output()
        try: 
            dictio = {}
            Symbol = st['symbol']
            dictio['Symbol'] = Symbol
            try:
                dictio['name'] = st['shortName']
            except: 
                dictio['name'] = st['longName']
                
            dictio['lastsale'] = np.round(st['regularMarketPrice'], 2)
            dictio['netchange'] = np.round(st['regularMarketChange'], 2)
            dictio['pctchange'] = np.round(st['regularMarketChangePercent'], 4)
            dictio['last_price'] = np.round(st['regularMarketPrice'], 2)
            dictio['pchange'] = np.round(st['regularMarketChangePercent'], 4)
            temp_df = pd.DataFrame(dictio, index = [0])
            stock_df = pd.concat([stock_df, temp_df]).drop_duplicates(subset = ['Symbol'])
            print(i, num)
        except: 
            pass
stock_df.reset_index(drop = True, inplace=True)
stock_df.columns = [col.upper() for col in stock_df.columns]
stock_df

19 535
19 536
19 537
19 538
19 539


,SYMBOL,NAME,LASTSALE,NETCHANGE,PCTCHANGE,LAST_PRICE,PCHANGE
0,IAI,iShares U.S. Broker-Dealers & S,153.28,0.85,0.5576,153.28,0.5576
1,VIRT,"Virtu Financial, Inc.",36.30,-0.73,-1.9714,36.30,-1.9714
2,LPCN,Lipocine Inc.,5.10,0.11,2.2044,5.10,2.2044
3,CTRE,"CareTrust REIT, Inc.",29.34,-0.09,-0.3058,29.34,-0.3058
4,ACVA,ACV Auctions Inc.,22.34,0.40,1.8232,22.34,1.8232
...,...,...,...,...,...,...,...
10641,ELTX,"Elicio Therapeutics, Inc.",4.84,0.03,0.6237,4.84,0.6237
10642,MOLN,Molecular Partners AG,5.72,0.19,3.4358,5.72,3.4358
10643,SCCF,Sachem Capital Corp. 7.125% Not,20.51,-0.17,-0.8394,20.51,-0.8394
10644,RILYM,"B. Riley Financial, Inc. - 6.37",23.25,-0.11,-0.4922,23.25,-0.4922


In [8]:
add_info = pd.read_csv('ticker_sectors.csv')
stocks = stock_df.merge(add_info.rename(columns = {'WEBSITE':'URL'}), how = 'left', on = 'SYMBOL')[nasdaq.drop(['MARKETCAP', 'IPOYEAR', 'VOLUME'], axis = 1).columns].dropna(subset = ['SYMBOL'])


In [42]:
add_info[add_info.SYMBOL=='ALLY']

,SYMBOL,COUNTRY,WEBSITE,INDUSTRY,SECTOR
2924,ALLY,United States,https://www.informatica.com,Software—Infrastructure,Technology


In [9]:
### Add exchanges names to the stocks dataframe
keys = []
for key, value in all_yh_tic.items():
    keys.append(key)
    stocks.loc[:,key] = np.where(stocks.SYMBOL.isin(value), 1, 0)
    print(key, stocks.shape)
stocks.loc[:,'NUM_EXCHANGES'] = stocks[keys].sum(1)

ft100 (10646, 12)
ft250 (10646, 13)
ibo (10646, 14)
n50 (10646, 15)
nfb (10646, 16)
nas (10646, 17)
dow (10646, 18)
other (10646, 19)
sp500 (10646, 20)


In [26]:
from pandas_datareader import data
from yfinance.multi import download
from functools import wraps
#https://stackoverflow.com/questions/78561896/installed-correct-version-of-panda-datareader-but-seem-to-be-getting-an-error-wh

def override_yahoo_behavior(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Extract the data_source parameter from the kwargs
        data_source = kwargs.get("data_source")
        # If data_source is not provided in kwargs, check args
        if data_source is None and len(args) > 1:
            data_source = args[1]
        # Check if the data_source is "yahoo" and return the custom behavior
        if data_source == "yahoo":
            # get the name, start, and end date from the kwargs or args
            name = kwargs.get("name")
            start_date = kwargs.get("start")
            end_date = kwargs.get("end")
            # If not in kwargs, check args
            if name is None and len(args) > 0:
                name = args[0]
            if start_date is None and len(args) > 2:
                start_date = args[2]
            if end_date is None and len(args) > 3:
                end_date = args[3]
            return download(tickers=name, start=start_date, end=end_date)
        # Otherwise, call the original function
        return func(*args, **kwargs)
    return wrapper

def hist_checker(ticker):
    '''
    Captures the history of the symbol provided. 
    It can be a ticker or a symbol like : '^GSPC' for S&P 500
    returns the historical data and the return of the index
    '''
    index_name = ticker# '^GSPC' # S&P 500
    start_datef = datetime.datetime.now() - datetime.timedelta(days=365*2)
    end_datef = datetime.date.today()
    data.DataReader = override_yahoo_behavior(data.DataReader)
    index_df = data.DataReader(index_name, "yahoo", start_datef, end_datef)
    index_df['Percent Change'] = index_df['Adj Close'].pct_change()
    # index_return = (index_df['Percent Change'] + 1).cumprod()[-1]
    index_return = (index_df['Percent Change'] + 1).cumprod().iloc[-1]
    index_df.columns = ['Adj Close', 'Close',	'High',	'Low', 'Open','Volume',	'Percent Change']
    index_df.reset_index(inplace = True)
    index_df.insert(loc=0, column='Symbol', value=ticker)
    index_df.columns = [col.upper() for col in index_df.columns]
    return index_df, index_return

def ticker_hist(tickers_list, pctreturn):
    stock_2yrs = rs_df = pd.DataFrame()
    returns_multiples = []
    data.DataReader = override_yahoo_behavior(data.DataReader)
    for ticker in tickers_list:
        clear_output(wait=True)
        # Download historical data as CSV for each stock (makes the process faster)
        try:
            df, stock_return = hist_checker(ticker)
            # Calculating returns relative to the market (returns multiple)
            returns_multiple = round((stock_return / pctreturn), 2)
            returns_multiples.extend([returns_multiple])
            stock_2yrs = pd.concat([stock_2yrs, df], sort =  False )
            # Creating dataframe of only top 30%
            rs_df = pd.DataFrame(list(zip(tickers_list, returns_multiples)), columns=['SYMBOL', 'RETURNS_MULTIPLE' ])
    #             rs_df['RS_Rating'] = rs_df.Returns_multiple.rank(pct=True) * 100
    #             rs_df = rs_df[rs_df.RS_Rating >= rs_df.RS_Rating.quantile(.70)]#         time.sleep(0.25)
        except Exception as E:
            print('ERROR:' , str(E))
            pass
        
    return stock_2yrs, returns_multiples, rs_df

In [27]:
def minervini_check(rs_df, stock_2yrs):
    '''
    Checking Minervini conditions of top 30% of stocks in given list
    '''
    minervi_df = pd.DataFrame(columns=['SYMBOL', "50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High"])
    rs_stocks = rs_df['SYMBOL']
    for stock in rs_stocks: 
        # mdf = pd.DataFrame()#columns=['Symbol', "RS_Rating", "50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High"])
        # time.sleep(0.5)
        print(stock)
        # clear_output(wait=True)
        try:
            df = stock_2yrs[stock_2yrs.SYMBOL == stock]#pdr.get_data_yahoo(stock, start_date, end_date)
            sma = [50, 150, 200]
            for x in sma:
                df.loc[:,"SMA_"+str(x)] = round(df['ADJ CLOSE'].rolling(window=x).mean(), 2)
            # Storing required values 
            currentClose = df["ADJ CLOSE"].values[-1]
            moving_average_50 = df["SMA_50"].values[-1]
            moving_average_150 = df["SMA_150"].values[-1]
            moving_average_200 = df["SMA_200"].values[-1]
            low_of_52week = round(min(df["LOW"][-260:]), 2)
            high_of_52week = round(max(df["HIGH"][-260:]), 2)
#             RS_Rating = round(rs_df[rs_df['Symbol']==stock].RS_Rating.tolist()[0])
            try:
                moving_average_200_20 = mean(df["SMA_200"].values[-20:])
            except Exception:
                moving_average_200_20 = 0
            # Condition 1: Current Price > 150 SMA and > 200 SMA
            condition_1 = currentClose > moving_average_150 > moving_average_200   
            # Condition 2: 150 SMA and > 200 SMA
            condition_2 = moving_average_150 > moving_average_200
            # Condition 3: 200 SMA trending up for at least 1 month
            condition_3 = moving_average_200 > moving_average_200_20      
            # Condition 4: 50 SMA> 150 SMA and 50 SMA> 200 SMA
            condition_4 = True if  moving_average_50 > moving_average_150 > moving_average_200 else False         
            # Condition 5: Current Price > 50 SMA
            condition_5 = currentClose > moving_average_50          
            # Condition 6: Current Price is at least 30% above 52 week low
            condition_6 = currentClose >= (1.3*low_of_52week)         
            # Condition 7: Current Price is within 25% of 52 week high
            condition_7 = currentClose >= (.75*high_of_52week)      
            # If all conditions above are true, add stock to exportList
            mdf =pd.DataFrame.from_dict({'SYMBOL': [stock],"50 Day MA": [moving_average_50]
                                            , "150 Day Ma": [moving_average_150], "200 Day MA": [moving_average_200]
                                            , "52 Week Low": [low_of_52week], "52 week High": [high_of_52week]}
                                           )
            if condition_1 : #and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7:
                mdf.loc[:,'minervis'] = 'Yes'
                print (stock + " fulfills Minervini's requirements")
            else: 
                mdf.loc[:,'minervis'] = 'No'
                print("{} does NOT fulfill Minervini's requirements".format(stock))
            minervi_df = minervi_df.dropna(axis=1, how='all')
            mdf = mdf.dropna(axis=1, how='all')
            minervi_df = pd.concat([minervi_df, mdf], axis = 0)
            
        except Exception as e:
            print (str(e))
            print(f"Could not gather data on {stock}")
            return
    minervi_df.columns = [col.upper() for col in minervi_df.columns]
    return minervi_df

In [40]:
import statistics
# Calculating the retunr for the indexes - 
# List of indexes = https://finance.yahoo.com/markets/world-indices/
sp500_hist, sp500_return = hist_checker('^GSPC')
dow_hist, dow_return = hist_checker('^DJI')
average = statistics.mean([dow_return, sp500_return])
print(dow_return, sp500_return, average)
ticker = r'boom|pypl|tsla|ally|googl|DGLY|HOUR|bito'

stock_2yrs, returns_multiples, rs_df = ticker_hist(stocks[stocks.SYMBOL.str.contains(ticker, case = False)].SYMBOL.unique(), average)
mdffinal = stocks[stocks.SYMBOL.isin(stock_2yrs.SYMBOL.unique())].merge(minervini_check(rs_df, stock_2yrs), on = 'SYMBOL', how = 'left')
mdffinal

[*********************100%***********************]  1 of 1 completed

GOOGL
GOOGL fulfills Minervini's requirements
DGLY
DGLY does NOT fulfill Minervini's requirements
BOOM
BOOM does NOT fulfill Minervini's requirements
HOUR
HOUR does NOT fulfill Minervini's requirements
PYPL
PYPL fulfills Minervini's requirements
ALLY
ALLY fulfills Minervini's requirements
TSLA
TSLA fulfills Minervini's requirements
BITO
BITO fulfills Minervini's requirements



/var/folders/97/pdf85q016hxcvxb1w3tvd3s00000gn/T/ipykernel_19566/2751217801.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"SMA_"+str(x)] = round(df['ADJ CLOSE'].rolling(window=x).mean(), 2)
/var/folders/97/pdf85q016hxcvxb1w3tvd3s00000gn/T/ipykernel_19566/2751217801.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"SMA_"+str(x)] = round(df['ADJ CLOSE'].rolling(window=x).mean(), 2)
/var/folders/97/pdf85q016hxcvxb1w3tvd3s00000gn/T/ipykernel_19566/2751217801.py:16: SettingWithCopyW

,SYMBOL,NAME,LASTSALE,NETCHANGE,PCTCHANGE,COUNTRY,SECTOR,INDUSTRY,URL,LAST_PRICE,...,dow,other,sp500,NUM_EXCHANGES,50 DAY MA,150 DAY MA,200 DAY MA,52 WEEK LOW,52 WEEK HIGH,MINERVIS
0,GOOGL,Alphabet Inc.,174.71,2.07,1.1990,NaN,NaN,NaN,NaN,174.71,...,0,0,0,1,169.40,170.10,164.98,127.90,191.75,Yes
1,DGLY,"Digital Ally, Inc.",0.84,0.09,12.2995,United States,Communication Services,Internet Content & Information,https://www.digitalallyinc.com,0.84,...,0,0,0,1,0.91,1.63,1.81,0.56,3.29,No
2,BOOM,DMC Global Inc.,8.01,0.00,0.0000,NaN,NaN,NaN,NaN,8.01,...,0,0,0,1,10.28,12.03,13.40,7.16,19.73,No
3,HOUR,"Hour Loop, Inc.",1.38,0.05,3.6992,United States,Consumer Cyclical,Internet Retail,https://www.hourloop.com,1.38,...,0,0,0,1,1.46,1.23,1.23,0.83,2.20,No
4,PYPL,"PayPal Holdings, Inc.",89.88,0.83,0.9377,Israel,Industrials,Building Products & Equipment,https://www.caesarstone.com,89.88,...,0,0,0,1,82.49,70.87,69.04,55.71,90.27,Yes
5,ALLY,Ally Financial Inc.,38.55,0.18,0.4691,United States,Technology,Software—Infrastructure,https://www.informatica.com,38.55,...,0,1,0,1,36.08,38.25,38.03,26.50,45.46,Yes
6,TSLA,"Tesla, Inc.",389.22,19.73,5.3398,NaN,NaN,NaN,NaN,389.22,...,0,0,0,1,284.07,233.98,219.03,138.80,389.49,Yes
7,BITO,ProShares Bitcoin ETF,26.12,0.63,2.4716,NaN,NaN,NaN,NaN,26.12,...,0,1,0,1,20.06,17.75,17.74,16.11,33.79,Yes


In [13]:
# FILTERS:
# Price
lower_price  =  0
higher_price = 60
# Percentage changes
lower_pct  = -200
higher_pct = 100
# Country
country = 'United States'

lower_price  = min_value(lower_price, stocks, 'LAST_PRICE')
higher_price = max_value(higher_price, stocks, 'LAST_PRICE')
lower_pct    = min_value(lower_pct, stocks, 'PCHANGE')
higher_pct   = max_value(higher_pct, stocks, 'PCHANGE')

stocksf = filter_stocks(mdffinal, lower_price, higher_price,lower_pct, higher_pct, country )
stocksf.sort_values(by = 'PCHANGE', ascending = False)

,SYMBOL,NAME,LASTSALE,NETCHANGE,PCTCHANGE,COUNTRY,SECTOR,INDUSTRY,URL,LAST_PRICE,...,dow,other,sp500,NUM_EXCHANGES,50 DAY MA,150 DAY MA,200 DAY MA,52 WEEK LOW,52 WEEK HIGH,MINERVIS
1,DGLY,"Digital Ally, Inc.",0.84,0.09,12.2995,United States,Communication Services,Internet Content & Information,https://www.digitalallyinc.com,0.84,...,0,0,0,1,0.91,1.63,1.81,0.56,3.29,No
3,HOUR,"Hour Loop, Inc.",1.38,0.05,3.6992,United States,Consumer Cyclical,Internet Retail,https://www.hourloop.com,1.38,...,0,0,0,1,1.46,1.23,1.23,0.83,2.20,No
5,ALLY,Ally Financial Inc.,38.55,0.18,0.4691,United States,Technology,Software—Infrastructure,https://www.informatica.com,38.55,...,0,1,0,1,36.08,38.25,38.03,26.50,45.46,Yes


SNCRL ESP  ARLP PTEN LBRT

In [14]:
def get_tickers_most_recent_divident(tickers):
    stock_info = pd.DataFrame()
    for ticker in tickers:
        try: 
            tickerdiv = si.get_dividends(ticker)
            tickerdiv = tickerdiv[tickerdiv.index == tickerdiv.index.max()].reset_index()
            tickerdiv.rename(columns = {'index':'Dividend_DT', 'ticker':'Symbol', 'dividend':'Dividend_AMT'}, inplace = True)
            stock_info = pd.concat([stock_info, tickerdiv], axis = 0)
        except Exception as E: 
            print("\t", E)
            continue
    stock_info.columns = [col.upper() for col in stock_info.columns]
    return stock_info.reset_index(drop = True)

In [15]:
ticker ='BBW'
one_stock_div  = stocks[stocks.SYMBOL ==ticker].merge(get_tickers_most_recent_divident([ticker]), on = 'SYMBOL', how = 'left')


/opt/anaconda3/envs/p312/lib/python3.12/site-packages/yahoo_fin/stock_info.py:693: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  frame.index = pd.to_datetime(frame.index, unit = "s")


In [16]:
broadc  = stocks[stocks.INDUSTRY == 'Broadcasting']
broadcf = filter_stocks(broadc, 0, 150,lower_pct, higher_pct, country ).sort_values(by = 'PCHANGE', ascending = False)
broadc.sort_values(by = 'NAME')

,SYMBOL,NAME,LASTSALE,NETCHANGE,PCTCHANGE,COUNTRY,SECTOR,INDUSTRY,URL,LAST_PRICE,...,ft100,ft250,ibo,n50,nfb,nas,dow,other,sp500,NUM_EXCHANGES
1967,SBIO,ALPS Medical Breakthroughs ETF,38.56,0.75,1.9931,United States,Communication Services,Broadcasting,https://www.iheartmedia.com,38.56,...,0,0,0,0,0,0,0,1,0,1
9983,ABLV,Able View Global Inc.,0.71,-0.08,-10.1266,United States,Communication Services,Broadcasting,https://www.nexstar.tv,0.71,...,0,0,0,0,0,1,0,0,0,1
4527,AZTA,"Azenta, Inc.",45.59,0.59,1.3111,United States,Communication Services,Broadcasting,https://www.fubo.tv,45.59,...,0,0,0,0,0,1,0,0,0,1
6103,BITC,Bitwise Funds Trust Bitwise Bit,74.34,1.67,2.2983,United States,Communication Services,Broadcasting,https://mediacoholding.com,74.34,...,0,0,0,0,0,0,0,1,0,1
9116,RECS,Columbia Research Enhanced Core,36.42,0.05,0.1375,United States,Communication Services,Broadcasting,https://www.gray.tv,36.42,...,0,0,0,0,0,0,0,1,0,1
3851,CUBE,CubeSmart,47.48,-0.05,-0.1052,United States,Communication Services,Broadcasting,https://bbgi.com,47.48,...,0,0,0,0,0,0,0,1,0,1
4182,ECCV,Eagle Point Credit Company Inc.,23.08,-0.12,-0.5051,United States,Communication Services,Broadcasting,https://www.cumulusmedia.com,23.08,...,0,0,0,0,0,0,0,1,0,1
6429,LANDO,Gladstone Land Corporation - 6.,22.28,0.02,0.0809,United States,Communication Services,Broadcasting,https://www.curiositystream.com,22.28,...,0,0,0,0,0,1,0,0,0,1
4250,PKBK,"Parke Bancorp, Inc.",23.03,-0.02,-0.0868,United States,Communication Services,Broadcasting,https://www.sagacom.com,23.03,...,0,0,0,0,0,1,0,0,0,1
6887,PHIO,Phio Pharmaceuticals Corp.,2.84,0.07,2.5271,United States,Communication Services,Broadcasting,https://www.gray.tv,2.84,...,0,0,0,0,0,1,0,0,0,1


In [50]:
mdffinal.SYMBOL.unique()

array(['GOOGL', 'DGLY', 'BOOM', 'HOUR', 'PYPL', 'ALLY', 'TSLA', 'BITO'],
      dtype=object)

In [ ]:
# recomd = get_tickers_information(mdf.Symbol.unique())
credentials = get_credentials()
recomd = get_tickers_information(credentials,['GOOGL', 'DGLY', 'ALLY'], get_quote)
bc = mdffinal.merge(recomd, on = 'SYMBOL', how = 'left')
bc.head()

TypeError: 'NoneType' object is not iterable

In [18]:
bc.sort_values(by = 'DIVIDEND_AMT', ascending = False).head()

,SYMBOL,NAME,LASTSALE,NETCHANGE,PCTCHANGE,COUNTRY,SECTOR,INDUSTRY,URL,LAST_PRICE,...,200 DAY MA,52 WEEK LOW,52 WEEK HIGH,MINERVIS,DIVIDEND_DT,DIVIDEND_AMT,PRICE,DIVIDENT_YIELD,ANALYST_RECOMDTN,DIVIDENT_RATE
7,BITO,ProShares Bitcoin ETF,26.12,0.63,2.4716,NaN,NaN,NaN,NaN,26.12,...,17.74,16.11,33.79,Yes,2024-12-02,0.996,26.12,46.66,,
5,ALLY,Ally Financial Inc.,38.55,0.18,0.4691,United States,Technology,Software—Infrastructure,https://www.informatica.com,38.55,...,38.03,26.50,45.46,Yes,2024-11-01,0.300,38.55,3.11,2.4 - Buy,1.2
0,GOOGL,Alphabet Inc.,174.71,2.07,1.1990,NaN,NaN,NaN,NaN,174.71,...,164.98,127.90,191.75,Yes,2024-09-09,0.200,174.71,0.46,1.6 - Buy,0.8
2,BOOM,DMC Global Inc.,8.01,0.00,0.0000,NaN,NaN,NaN,NaN,8.01,...,13.40,7.16,19.73,No,2020-03-30,0.125,8.01,,2.7 - Hold,
1,DGLY,"Digital Ally, Inc.",0.84,0.09,12.2995,United States,Communication Services,Internet Content & Information,https://www.digitalallyinc.com,0.84,...,1.81,0.56,3.29,No,NaT,NaN,NaN,NaN,NaN,NaN


# visualization

https://towardsdatascience.com/python-how-to-get-live-market-data-less-than-0-1-second-lag-c85ee280ed93


In [19]:
def stockplot(ticker, datf, field, plttype='d', pltwidth=800, pltheigth=300 ):
    inc = datf.Close > datf.Open
    dec = datf.Open > datf.Close
    if plttype == 'd':
        w = 100000
        title = " Last day ({}) behavior for {} stocks.".format(datf.Date.max().strftime('%Y%m%d'), ticker)
    elif plttype == 'pd':
        w = 60*60*10000 
        title = " Daily behavior comparison between {0} and {1} for {2} stocks".format(datf.Date.min().strftime('%Y%m%d'), datf.Date.max().strftime('%Y%m%d'), ticker)
    elif plttype == 'w':
        w = 60*60*10000 
        title = " Daily behavior comparison between {0} and {1} for {2} stocks".format(datf.Date.min().strftime('%Y%m%d'), datf.Date.max().strftime('%Y%m%d'), ticker)
    elif plttype == 'y':
        w = 60*60*60*10000 
        title = " Monthly behavior between {0} and {1} for {2} stocks".format(datf.Date.min().strftime('%Y%m%d'), datf.Date.max().strftime('%Y%m%d'), ticker)
    else:
        w = 1000
        dat = df.Date.max().strftime('%Y%m%d')

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

    p = figure(x_axis_type="datetime", tools=TOOLS, width=pltwidth, height=pltheigth, title = title)
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3

    p.segment(datf[field], datf.High, datf[field], datf.Low, color="black")
    p.vbar(datf[field][inc], w, datf.Open[inc], datf.Close[inc], fill_color="#D5E1DD", line_color="green")
    p.vbar(datf[field][dec], w, datf.Open[dec], datf.Close[dec], fill_color="#D5E1DD", line_color="red")
    return p 

In [20]:
## https://github.com/ndepaola/bokeh-candlestick/blob/master/candlestick_plot.py
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.models.formatters import DatetimeTickFormatter

def stockplot2(ticker, df, field, plttype='d', pltwidth=800, pltheigth=300 ):
    # width and title of the bars 
    if plttype == 'd':
        w = 100000
        title = " Last day ({}) behavior for {} stocks.".format(df.Date.max().strftime('%Y%m%d'), ticker)
    elif plttype == 'pd':
        w = 60*120*10000
        title = " Last two days behavior -  {0} and {1} - for {2} stocks".format(df.Date.min().strftime('%Y%m%d'), df.Date.max().strftime('%Y%m%d'), ticker)
    elif plttype == 'w':
        w = 60*60*10000 
        title = " Daily behavior between {0} and {1} for {2} stocks".format(df.Date.min().strftime('%Y%m%d'), df.Date.max().strftime('%Y%m%d'), ticker)
    elif plttype == 'y':
        w = 60*60*60*10000 
        title = " Last year behavior comparison between {0} and {1} for {2} stocks".format(df.Date.min().strftime('%Y%m%d'), df.Date.max().strftime('%Y%m%d'), ticker)
    else:
        w = 1000
        dat = df.Date.max().strftime('%Y%m%d')

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    inc = df.Close > df.Open
    dec = df.Open > df.Close
    # Colour scheme for increasing and descending candles
    INCREASING_COLOR = '#31a354'#"#D5E1DD"
    DECREASING_COLOR = '#de2d26' #"#D5F1DD"
    # Defining data sources
    inc_source = ColumnDataSource(data=dict(
                                            x1=df.index[inc],
                                            open1=df.Open[inc],
                                            close1=df.Close[inc],
                                            high1=df.High[inc],
                                            low1=df.Low[inc],
                                            Date1=df[field][inc]
                                            )
                                 )
    dec_source = ColumnDataSource(data=dict(
                                            x2 = df.index[dec],
                                            open2 = df.Open[dec],
                                            close2 = df.Close[dec],
                                            high2 = df.High[dec],
                                            low2 = df.Low[dec],
                                            Date2 = df[field][dec]
                                            )
                                 )
    # Plot candles
    # Initializing figure
    fig = figure(x_axis_type="datetime", tools=TOOLS, width=pltwidth,plot_height=pltheigth, title = title)
    fig.background_fill_color = "#636363"#"black"
    fig.xaxis.major_label_orientation = pi/4
    fig.grid.grid_line_alpha=0.3
    # Adding segments
    fig.segment(df[field], df.High, df[field], df.Low, color="black")
    # Adding bars
    d1 = fig.vbar('Date1', w, 'open1', 'close1', source=inc_source, color=INCREASING_COLOR, line_color = 'black')
    d2 = fig.vbar('Date2', w, 'open2', 'close2', source=dec_source, color=DECREASING_COLOR,  line_color="black")
    # Adding hover tools
    fig.add_tools(HoverTool(
                            renderers=[d1],
                            tooltips=[
                                        ("Open", "$@open1"),
                                        ("High", "$@high1"),
                                        ("Low", "$@low1"),
                                        ("Close", "$@close1"),
                                        #             ("Date", "@Date1{%Y-%m-%d 3N}")
                                        ]
                            , formatters={
                            'Date1': 'datetime'
                            }))
    fig.add_tools(HoverTool(
                            renderers=[d2],
                            tooltips=[
                                        ("Open", "$@open2"),
                                        ("High", "$@high2"),
                                        ("Low", "$@low2"),
                                        ("Close", "$@close2"),
                                        #             ("Date", "@Date2{%Y-%m-%d%N}")
                                        ]
                            , formatters={'Date2': 'datetime'}
                            )
                 )
    return fig 

In [21]:
tickr = 'BITO'
# Last day data
dfd = yf.download(tickers = tickr, period = '1d', interval = '1m').reset_index()
dfd.columns = dfd.columns.get_level_values(0) 
dfd["Date"] = pd.to_datetime(dfd["Datetime"]).dt.date
dfd["Time"] = pd.to_datetime(dfd["Datetime"]).dt.time
# Previous day data 
dfpday = yf.download(tickers = tickr, period = '1d', interval = '1d').reset_index().rename(columns = {'index':'Datetime'}) 
dfpday.columns = dfpday.columns.get_level_values(0) 
dfpday["Date"] = pd.to_datetime(dfpday["Date"]).dt.date
# dfpday["Time"] = pd.to_datetime(dfpday["Datetime"]).dt.time
# dfpday["Weekday"] = pd.to_datetime(dfpday["Date"]).dt.day_name()
# Weekly data 
dfw = yf.download(tickers = tickr, period ='5d', interval ='1d').reset_index() 
dfw.columns = dfw.columns.get_level_values(0) 
dfw = dfw.merge(pd.DataFrame({'Date':pd.date_range(start = dfw.Date.min().strftime('%Y%m%d'), end = dfw.Date.max().strftime('%Y%m%d'), 
                      freq = '1D')}), on = 'Date', how = 'outer').sort_values(by = 'Date').ffill()
dfw["Weekday"] = pd.to_datetime(dfw["Date"]).dt.day_name()
# Last Year
dfy = yf.download(tickers = tickr, period = '5y', interval = '1mo').reset_index() 
dfy.columns = dfy.columns.get_level_values(0) 
dfy = dfy[dfy.Date <= dfy.Date.max().replace(day=1)]

# f1 = stockplot2(tickr, dfd, "Time", 'd')
# f2 = stockplot2(tickr, dfpday, "Date", 'pd')
# f3 = stockplot2(tickr, dfw, "Date", 'w')
# f4 = stockplot2(tickr, dfy, "Date", 'y')
# show(column(row(f1), row(f3, f2)))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [22]:
output_notebook()
f1 = stockplot(tickr, dfd, "Time", 'd')
f2 = stockplot(tickr, dfpday, "Date", 'pd')
f3 = stockplot(tickr, dfw, "Date", 'w')
f4 = stockplot(tickr, dfy, "Date", 'y')
show(column(row(f1, f2), row(f3, f4)))

Loading BokehJS ...

In [23]:
final = datetime.datetime.now()
print(final - current_time)

0:00:28.866523


# CHECK THIS SITE FOR A MORE COMPLEX DASHBOARD 


https://medium.com/analytics-vidhya/building-a-technical-analysis-chart-with-python-17107b78b297

https://blog.quantinsti.com/stock-market-data-analysis-python/

predictors:
    https://www.section.io/engineering-education/stock-price-prediction-using-python/

In [ ]:
# yf.download('Boom' ,start='2021-1', period='1m')

a = yf.download(tickers = 'Boom', start='2021-11-05',period = '1d').reset_index()

In [ ]:
a

In [ ]:
a[(a.Open>42.7)&(a.Open<42.8)]

In [ ]:
a

In [ ]:
dfd

In [ ]:
import smtplib

username = "jpinzon@hotmail.com"
password = ""

vtext = "3058078557@vtext.com"
message = "this is the message to be sent"

msg = """From: %s
To: %s
Subject: text-message
%s""" % (username, vtext, message)

# server = smtplib.SMTP('smtp.hotmail.com',587)
# server.starttls()
# server.login(username,password)
# server.sendmail(username, vtext, msg)
# server.quit()

In [ ]:
import email
import smtplib
from email.mime.text import MIMEText
from email.header import Header

body = u'some crzy code'
subject = u'some crzy code2double email '

msg = MIMEText(body,'plain','UTF-8')
msg['Subject'] = Header(subject,'UTF-8')
header_charset = 'UTF-8'
#msg['Subject'] = Header(str(subject,'UTF-8'), header_charset)

# msg['From'] = "jpinzon@hotmail.com"
# msg['To'] = "jpinzon@hotmail.com.com"
# msg['Cc'] = "3058078557@vtext.com"


s = smtplib.SMTP("smtp.live.com",587)
s.ehlo() # Hostname to send for this command defaults to the fully qualified domain name of the local host.
s.starttls() #Puts connection to SMTP server in TLS mode
s.ehlo()
s.login('jpinzon@hotmail.com', 'M8w10j9r4')

s.sendmail("jpinzon@hotmail.com", ['jorgecoral@hotmail.com',"3058078557@vtext.com"], msg.as_string())

s.quit()

In [ ]:
import tkinter as tk

root = tk.Tk()

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

In [ ]:
screen_height

In [ ]:
screen_width


In [ ]:
'''Get the latest value of all stocks FROM Nasdaq.com
https://github.com/shilewenuw/get_all_tickers/issues/12

Available exchanges: nasdaq, amex, nyse
'''
headers = {'authority': 'api.nasdaq.com',
            'accept': 'application/json, text/plain, */*',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
            'origin': 'https://www.nasdaq.com',
            'sec-fetch-site': 'same-site',
            'sec-fetch-mode': 'cors',
            'sec-fetch-dest': 'empty',
            'referer': 'https://www.nasdaq.com/',
            'accept-language': 'en-US,en;q=0.9',
            }

dparams = (('tableonly', 'true'),
            ('limit', '250'),
            ('offset', '0'),
            ('download', 'true')
            )    
# if exchange != None:
#     dparams = dparams + (('exchange',exchange),)
# dparams = dparams + (('exchange','nyse'),)
r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers = headers, params = dparams)

In [ ]:
stocksj = r.json()['data']
stocksdf = pd.DataFrame(stocksj['rows'], columns = stocksj['headers'])
# stocksdf = stocksdf[~stocksdf['symbol'].str.contains("\.|\^")]

In [ ]:
stocksdf[stocksdf.symbol.str.contains('^BIT')]

In [ ]:
def __exchange2df(exchange):
    # response = requests.get('https://old.nasdaq.com/screening/companies-by-name.aspx', headers=headers, params=params(exchange))
    # data = io.StringIO(response.text)
    # df = pd.read_csv(data, sep=",")
    params = (
        ('tableonly', 'false'),
        ('limit', '250'),
        ('offset', '0'),
        ('download', 'true'),
        ('exchange', exchange)
    )

    r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers=headers, params=params)
    data = r.json()['data']
    df = pd.DataFrame(data['rows'], columns=data['headers'])
    return df

In [ ]:
df = __exchange2df(None)
# nasdaq, amex, nyse
df[df.symbol.str.contains('^BIT')]

In [ ]:
df[df.symbol.str.contains('ALLY')]

In [ ]:
yf.download(tickers = 'BITO', period = '2d', interval = '1d').reset_index()#.rename(columns = {'index':'Datetime'}) 

In [ ]:
pdr.get_data_yahoo('BITO', '2024-05-22', '2024-05-24')

In [ ]:
ticker ='BITO' 
credentials = get_credentials()

get_tickers_information(credentials, [ticker], get_quote)

In [ ]:
q = get_quote(['BITO'], credentials)
q

In [ ]:
bq = get_quote(['BITO'], credentials)[0]
print(bq['symbol'], bq['longName'], bq['regularMarketPrice'], np.round(bq['regularMarketChange'], 2), np.round(bq["regularMarketChangePercent"], 3), bq["region"])

In [ ]:
df[df.symbol.str.contains('ALLY')]

In [ ]:
bq = get_quote(['ALLY'], credentials)[0]
print(bq['symbol'], bq['longName'], bq['regularMarketPrice'], np.round(bq['regularMarketChange'], 2), np.round(bq["regularMarketChangePercent"], 3), bq["region"])

In [32]:
a = 1
b = 2
pd.DataFrame([[a,b]], columns= ['A', 'B'])

,A,B
0,1,2


In [34]:
import sqlite3
conn = sqlite3.connect("./stock_analysis/static/stock_data.db")

In [38]:
pd.read_sql('select * FROM stocks', conn)

,SYMBOL,NAME,LASTSALE,NETCHANGE,PCTCHANGE,COUNTRY,SECTOR,INDUSTRY,URL,LAST_PRICE,...,ft100,ft250,ibo,n50,nfb,nas,dow,other,sp500,NUM_EXCHANGES
0,PTIR,GraniteShares 2x Long PLTR Dail,136.69,14.91,12.2434,None,None,None,None,136.69,...,0,0,0,0,0,1,0,0,0,1
1,LFAK,Stone Ridge 2052 Longevity Inco,14.61,0.03,0.1735,United States,Industrials,Building Products & Equipment,https://www.corporate.carrier.com,14.61,...,0,0,0,0,0,0,0,1,0,1
2,CAF,Morgan Stanley China A Share Fu,12.63,0.18,1.4458,None,None,None,None,12.63,...,0,0,0,0,0,0,0,1,0,1
3,CRKN,Crown Electrokinetics Corp.,0.25,0.01,4.6199,United States,Financial Services,Shell Companies,https://alchemyinvest.co,0.25,...,0,0,0,0,0,1,0,0,0,1
4,FIZZ,National Beverage Corp.,46.32,-3.24,-6.5375,None,None,None,None,46.32,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10641,VSME,VS Media Holdings Limited,1.05,0.02,1.9417,None,None,None,None,1.05,...,0,0,0,0,0,1,0,0,0,1
10642,WDIV,SPDR S&P Global Dividend,65.61,-0.27,-0.4156,None,None,None,None,65.61,...,0,0,0,0,0,0,0,1,0,1
10643,FESM,Fidelity Covington Trust Fideli,35.34,0.10,0.2838,None,None,None,None,35.34,...,0,0,0,0,0,0,0,1,0,1
10644,JAVA,JPMorgan Active Value ETF,67.49,-0.10,-0.1479,United States,Technology,Semiconductor Equipment & Materials,https://www.uct.com,67.49,...,0,0,0,0,0,0,0,1,0,1
